In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
from typing import List, Tuple, Dict
import math

In [3]:
myint = tf.int32
myfloat = tf.float32

In [4]:
learning_rate = 0.001
batch_size  =128
num_epoch = 1000

In [5]:
shape_x = np.array([None, 32, 32, 3])
shape_y = np.array([None])
shape_1st_convolution = np.array([5, 5, 3, 32]) # with pad 2 stride 1
shape_1st_layer = np.array([32, 32, 32])
shape_2nd_pool = np.array([2, 2]) # with stride 2
shape_2nd_layer = np.array([16, 16, 32])
# 3rd normalization
shape_4th_convolution = np.array([5, 5, 32, 64]) # with pad 2 stride 1
shape_4th_layer = np.array([16, 16, 64])
shape_5th_pool = np.array([2, 2])  # with stride 2
shape_5th_layer = np.array([8, 8, 64])
# 6th normalization
shape_6th_dence = np.array([4096, 1024])  # 8*8*64
shape_6th_layer = np.array([1024])
shape_7th_weight = np.array([1024, 10])
shape_7th_layer = np.array([10])  # output

In [6]:
def assert_shape(target: np.ndarray, shape):
    if isinstance(shape, list):
        shape = np.array(shape)
    
    assert isinstance(target, np.ndarray), "target is not ndarray"
    assert isinstance(shape, np.ndarray), "shape is not ndarray"
    
    target_shape = target.shape
    assert len(target_shape) == len(shape), 'dimension is not equal'
    for (i, s) in enumerate(shape):
        assert target_shape[i] == s, "size of target[{}] is not {}".format(i, s)

In [7]:
class DataLoader:
    
    def __init__(self):
        self.data, self.labels = self._load_training_data()
        self.test_data, self.test_labels = self._load_test_data()
        self.epoch = 0
        self.step = 0
        
    def num_step(self, batch_size: int) -> int:
        return math.floor(len(self.data)/batch_size)
        
    def next_batch(self, batch_size: int):
        data_size = len(self.data)
        next_step = self.step + 1
        if next_step * batch_size > data_size:
            self.epoch += 1
            self.step = 0
        batch_data = self.data[self.step*batch_size:next_step*batch_size]
        batch_labels = self.labels[self.step*batch_size:next_step*batch_size]
        self.step += 1
        return batch_data, batch_labels
        
    def _load_training_data(self):
        path = 'data/cifar-10-batches-py/'
        file_format = 'data_batch_{}'
        train_data = []
        train_labels = []
        with open(path + file_format.format(1), 'r') as f:
            for i in range(5):
                data, labels = self._get_data_from_file(path + file_format.format(i+1))
                train_data.extend(self._get_reshaped_data(data))
                train_labels.extend(labels)
        return np.array(train_data), np.array(train_labels)
    
    def _load_test_data(self):
        filepath = 'data/cifar-10-batches-py/test_batch'
        data, labels = self._get_data_from_file(filepath)
        data = self._get_reshaped_data(data)
        return data, np.array(labels)
    
    def _get_data_from_file(self, filepath):
        f = open(filepath, 'rb')
        raw_data = pickle.load(f, encoding='bytes')
        data, labels = self._get_data_and_labels(raw_data)
        return data, labels
    
    def _get_data_and_labels(self, dict_data):
        data = dict_data[b'data']
        labels = dict_data[b'labels']
        return data, labels
    
    def _get_reshaped_data(self, data):
        # 元データのshape check
        original_data_shape = [10000, 32*32*3]
        assert_shape(data, original_data_shape)

        reshaped_data_shape = [10000, 32, 32, 3]
        reshaped_data = np.reshape(data, [10000, 3, 32, 32])
        reshaped_data = np.transpose(reshaped_data, [0, 2, 3, 1])

        # 修正したデータのshape check
        assert_shape(reshaped_data, reshaped_data_shape)
        return reshaped_data

In [8]:
datasource = DataLoader()

In [9]:
print(datasource.data.shape)
print(datasource.labels.shape)
print(datasource.test_labels.shape)
print(datasource.num_step(batch_size))

(50000, 32, 32, 3)
(50000,)
(10000,)
390


# network

In [10]:
class CNN:
    def __init__(self):
        self.x = tf.placeholder(dtype=myfloat, shape=shape_x, name='x')
        self.y = tf.placeholder(dtype=myint, shape=shape_y, name='y')
        self.y_onehot = tf.one_hot(indices=tf.cast(self.y, myint), depth=10)
        self.is_training = tf.placeholder(dtype=bool, name='is_training')
        l_1 = tf.layers.conv2d(inputs=self.x,
                                    filters=shape_1st_convolution[3],
                                    kernel_size=shape_1st_convolution[0:2],
                                    padding='same',
                                    activation=tf.nn.relu)
        l_2 = tf.layers.max_pooling2d(inputs=l_1, pool_size=[2, 2], strides=2)
        l_4 = tf.layers.conv2d(inputs=l_2,
                                     filters=shape_4th_convolution[3] ,
                                     kernel_size=shape_4th_convolution[0:2],
                                     padding='same',
                                     activation=tf.nn.relu)
        l_5 = tf.layers.max_pooling2d(inputs=l_4, pool_size=[2, 2], strides=2)
        l_5_flat = tf.reshape(l_5, [-1, shape_6th_dence[0]])
        l_6 = tf.layers.dense(inputs=l_5_flat, units=shape_6th_dence[1], activation=tf.nn.relu)
        l_6 = tf.layers.dropout(inputs=l_6,
                                     rate=0.4,
                                     training=self.is_training)
        l_7 = tf.layers.dense(inputs=l_6, units=shape_7th_weight[1])
        self.logits = l_7

        self.predicted_classes = tf.argmax(input=self.logits, axis=1, output_type=myint)
        probs = tf.nn.softmax(self.logits)

In [11]:
cnn = CNN()

In [12]:
print(cnn.logits.get_shape())
print(cnn.y.get_shape())

(?, 10)
(?,)


In [13]:
crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=cnn.y, logits=cnn.logits)
loss_op = tf.reduce_mean(crossent)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op = optimizer.minimize(loss_op)
correct = tf.equal(cnn.predicted_classes, cnn.y)
acc_op = tf.reduce_mean(tf.cast(correct, myfloat))

In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(num_epoch):
        step_size = datasource.num_step(batch_size)
        for s in range(step_size):
            data, labels = datasource.next_batch(batch_size)
            fd = {
                cnn.x: data,
                cnn.y: labels,
                cnn.is_training: True
            }
            loss, _ = sess.run([loss_op, train_op], feed_dict=fd)
            #if s % 10 == 0:
            #    print('{}/{} step, loss: {}'.format(s, step_size, loss))
        acc = sess.run(acc_op, feed_dict={
            cnn.x: datasource.test_data,
            cnn.y: datasource.test_labels,
            cnn.is_training: False
        })
        print('{}/{} epoch, loss: {}, acc: {}'.format(i, num_epoch, loss, acc))

0/390 step, loss: 66.08597564697266
10/390 step, loss: 2.3134851455688477
20/390 step, loss: 2.299135208129883
30/390 step, loss: 2.282447338104248
40/390 step, loss: 2.293429374694824
50/390 step, loss: 2.284641981124878
60/390 step, loss: 2.2745251655578613
70/390 step, loss: 2.2338578701019287
80/390 step, loss: 2.216722011566162


KeyboardInterrupt: 

In [ ]:
loss.shape